In [1]:
import sqlite3
import pandas as pd 
from datetime import datetime

conexao = sqlite3.connect('../pagina/db.sqlite3')

df = pd.read_sql("select * from ingressar_paciente", conexao)
df.columns = df.columns.str.upper()

def calcular_idade(linha):
    idade = (datetime.now() - pd.to_datetime(linha['NASCIMENTO'])).days // 365

    return idade

df['IDADE'] = df.apply(calcular_idade, axis=1, result_type='expand')

df

,ID,NOME,SOBRENOME,IDADE,NASCIMENTO,ALTURA,PESO,SEXO,PRESSAO_ARTERIAL,GLICEMIA,FREQUENCIA_CARDIACA
0,1,JOSE,Lopes,42,1981-10-07,1.89,109.10,M,107/93,46,89.0
1,2,JOAO,Nascimento,87,1936-10-28,1.58,96.95,M,105/135,191,87.0
2,3,ANTONIO,Almeida,86,1937-07-02,1.67,91.12,M,184/117,132,84.0
3,4,FRANCISCO,Martins,73,1950-12-03,2.03,66.85,M,127/71,152,88.0
4,5,CARLOS,Monteiro,23,2000-05-24,2.01,89.65,M,93/119,237,83.0
...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,LUINE,Monteiro,46,1977-10-18,1.59,62.28,F,90/32,186,98.0
19996,19997,SILBENE,Martins,14,2009-11-04,1.73,102.45,F,157/43,139,103.0
19997,19998,GERSIANE,Barros,54,1969-07-20,2.07,96.75,F,181/96,79,99.0
19998,19999,DALVANEIDE,Mendes,64,1959-06-09,1.72,53.31,F,181/55,103,67.0


In [2]:
df2 = df.drop('ID', axis=1)

In [3]:
def indice_massa_corporal(linha):
    peso = float(linha['PESO'])
    altura = float(linha['ALTURA'])
    imc = round(peso / (altura**2), 1)

    if imc < 18.5:
        return 'SUBNUTRIDO', imc
    
    if 18.5 <= imc and imc <= 24.9:
        return 'NORMAL', imc
    
    if 25 <= imc and imc <= 29.9:
        return 'SOBREPESO', imc
    
    if 30 <= imc and imc <= 34.9:
        return 'OBESIDADE I', imc
    
    if 35 <= imc:
        return 'OBESIDADE II', imc
    
    return 'ERRO', imc

df2[['DESCRICAO_IMC', 'IMC']] = (df2.apply(indice_massa_corporal, axis=1, result_type='expand'))
df2.columns

Index(['NOME', 'SOBRENOME', 'IDADE', 'NASCIMENTO', 'ALTURA', 'PESO', 'SEXO',
       'PRESSAO_ARTERIAL', 'GLICEMIA', 'FREQUENCIA_CARDIACA', 'DESCRICAO_IMC',
       'IMC'],
      dtype='object')

In [4]:
def separa_antes(linha):
    pressao_sil = str(linha['PRESSAO_ARTERIAL'])[:3]

    if pressao_sil.endswith('/'):
        return pressao_sil[:2]
    
    return pressao_sil

df2['PRESSAO_SISTOLICA'] = df2.apply(separa_antes, axis=1)
df2



,NOME,SOBRENOME,IDADE,NASCIMENTO,ALTURA,PESO,SEXO,PRESSAO_ARTERIAL,GLICEMIA,FREQUENCIA_CARDIACA,DESCRICAO_IMC,IMC,PRESSAO_SISTOLICA
0,JOSE,Lopes,42,1981-10-07,1.89,109.10,M,107/93,46,89.0,OBESIDADE I,30.5,107
1,JOAO,Nascimento,87,1936-10-28,1.58,96.95,M,105/135,191,87.0,OBESIDADE II,38.8,105
2,ANTONIO,Almeida,86,1937-07-02,1.67,91.12,M,184/117,132,84.0,OBESIDADE I,32.7,184
3,FRANCISCO,Martins,73,1950-12-03,2.03,66.85,M,127/71,152,88.0,SUBNUTRIDO,16.2,127
4,CARLOS,Monteiro,23,2000-05-24,2.01,89.65,M,93/119,237,83.0,NORMAL,22.2,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,LUINE,Monteiro,46,1977-10-18,1.59,62.28,F,90/32,186,98.0,NORMAL,24.6,90
19996,SILBENE,Martins,14,2009-11-04,1.73,102.45,F,157/43,139,103.0,OBESIDADE I,34.2,157
19997,GERSIANE,Barros,54,1969-07-20,2.07,96.75,F,181/96,79,99.0,NORMAL,22.6,181
19998,DALVANEIDE,Mendes,64,1959-06-09,1.72,53.31,F,181/55,103,67.0,SUBNUTRIDO,18.0,181


In [5]:
def separa_depois(linha):
    pressao_dias = str(linha['PRESSAO_ARTERIAL']).replace(linha['PRESSAO_SISTOLICA'], '').replace('/', '')
    return pressao_dias

df2['PRESSAO_DIASTOLICA'] = df2.apply(separa_depois, axis=1)
df2

,NOME,SOBRENOME,IDADE,NASCIMENTO,ALTURA,PESO,SEXO,PRESSAO_ARTERIAL,GLICEMIA,FREQUENCIA_CARDIACA,DESCRICAO_IMC,IMC,PRESSAO_SISTOLICA,PRESSAO_DIASTOLICA
0,JOSE,Lopes,42,1981-10-07,1.89,109.10,M,107/93,46,89.0,OBESIDADE I,30.5,107,93
1,JOAO,Nascimento,87,1936-10-28,1.58,96.95,M,105/135,191,87.0,OBESIDADE II,38.8,105,135
2,ANTONIO,Almeida,86,1937-07-02,1.67,91.12,M,184/117,132,84.0,OBESIDADE I,32.7,184,117
3,FRANCISCO,Martins,73,1950-12-03,2.03,66.85,M,127/71,152,88.0,SUBNUTRIDO,16.2,127,71
4,CARLOS,Monteiro,23,2000-05-24,2.01,89.65,M,93/119,237,83.0,NORMAL,22.2,93,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,LUINE,Monteiro,46,1977-10-18,1.59,62.28,F,90/32,186,98.0,NORMAL,24.6,90,32
19996,SILBENE,Martins,14,2009-11-04,1.73,102.45,F,157/43,139,103.0,OBESIDADE I,34.2,157,43
19997,GERSIANE,Barros,54,1969-07-20,2.07,96.75,F,181/96,79,99.0,NORMAL,22.6,181,96
19998,DALVANEIDE,Mendes,64,1959-06-09,1.72,53.31,F,181/55,103,67.0,SUBNUTRIDO,18.0,181,55


In [6]:
df2.to_excel('../database//resultado.xlsx', index=False)